In [15]:
import os
import csv
from fuzzywuzzy import fuzz


c:\Users\Admin\.conda\envs\Cybersec_project\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
malicious_domains = ['wikileaks']

In [ ]:
def detect_malicious_users(dataset_path, malicious_domains):
    malicious_users = set()
    with open(os.path.join(dataset_path, "http.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            url = row[4]
            for m in malicious_domains:
                if m in url:
                    user_id = row[2]
                    malicious_users.add(user_id)
                    break
    return malicious_users

# Usage
malicious_users = detect_malicious_users('Insider threat dataset/r5.2', ['wikileaks'])
malicious_users


KEW0198
DAS1320
KEW0198
KEW0198
DAS1320
EPG1196
KBC1390
GFM1815
KBC1390
EPG1196
SAF1942
KBC1390
PBC0077
SAF1942
PBC0077
PBC0077
ALT1465


KeyboardInterrupt: 

## Email fuzzy match (shelved for now)

In [13]:
with open('Insider threat dataset/r5.2/email.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        print(row)
        if i==5:
            break

['id', 'date', 'user', 'pc', 'to', 'cc', 'bcc', 'from', 'activity', 'size', 'attachments', 'content']
['{N9X0-P3SX99UT-3623QQUN}', '01/02/2010 06:49:35', 'KMC1934', 'PC-2683', 'Harding.Alden.Lester@dtaa.com;Ora.Clementine.Lester@dtaa.com', 'Remedios.Kylynn.Weaver@dtaa.com;Hamilton.Palmer.Cortez@dtaa.com', 'Karina.Melanie.Collins@dtaa.com', 'Karina.Melanie.Collins@dtaa.com', 'Send', '22578', '', 'lost six speculated individual theoren in ontario pink modeled speculated role most canadian thirty failed including represent shootout owners professional sonne proving around 108 run pled williamson per popular preceded']
['{F5G8-U4KL36AB-1579CZGX}', '01/02/2010 06:50:35', 'KMC1934', 'PC-2683', 'Harding.Alden.Lester@dtaa.com;Ora.Clementine.Lester@dtaa.com', 'Remedios.Kylynn.Weaver@dtaa.com;Hamilton.Palmer.Cortez@dtaa.com', '', 'Karina.Melanie.Collins@dtaa.com', 'Receive', '22578', '', 'lost six speculated individual theoren in ontario pink modeled speculated role most canadian thirty failed i

In [ ]:
# Function for fuzzy matching
def is_self_email(sender, recipient, threshold=65):
    return fuzz.ratio(sender.lower(), recipient.lower()) >= threshold

# Function to flag emailing oneself clearly
def flag_self_emailing(dataset_path, threshold=85):
    flagged_users =set()
    csv_filepath = os.path.join(dataset_path, 'email.csv')
    with open(csv_filepath, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Skip header row explicitly

        for row in reader:
            sender = row[7].strip()
            recipients = row[4].split(';') + row[5].split(';') + row[6].split(';')
            recipients = [email.strip() for email in recipients if email.strip() != '']

            sender_domain = sender.split('@')[-1]

            for recipient in recipients:
                recipient_domain = recipient.split('@')[-1]

                # Only flag external (personal domain) matches
                if recipient_domain != sender_domain:
                    if is_self_email(sender.split('@')[0], recipient.split('@')[0], threshold):
                        print(f"Flagged: {sender} -> {recipient}")
                        flagged_users.add(row[2])

    return flagged_users

# Usage (clearly):
dataset_path = 'Insider threat dataset\\r5.2'
results = flag_self_emailing(dataset_path)


KeyboardInterrupt: 

In [ ]:
answers_path = 'Insider threat dataset\\answers\\r5.2-4' 
for f in os.listdir(answers_path):
    with open(os.path.join(answers_path, f), 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[0]=='email':
                sender = row[8].strip()
                recipients = row[5].split(';') + row[6].split(';') + row[7].split(';')
                recipients = [email.strip() for email in recipients if email.strip() != '']
                for recipient in recipients:
                    r = fuzz.ratio(sender.split('@')[0], recipient.split('@')[0])
                    print(f"{sender} -> {recipient}, {r}")
            


# other PC login

In [23]:
def get_user_logon_data(user_id, dataset_path):
    logon_data = []
    with open(os.path.join(dataset_path, "logon.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[2] == user_id:
                logon_data.append(row)
    return logon_data

def get_user_pc(logon_data):
    pc_dict = {}
    for row in logon_data:
        pc_dict[row[3]] = 1 + pc_dict.get(row[3], 0)
    user_pc = max(pc_dict, key=pc_dict.get)
    return user_pc

def check_other_pc_login(logon_data):
    user_pc = get_user_pc(logon_data)
    for row in logon_data:
        if row[3] != user_pc:
            return True
    return False

def get_other_pc_users(dataset_path):
    other_pc_users = set()
    with open(os.path.join(dataset_path, "logon.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            user_id = row[2]
            logon_data = get_user_logon_data(user_id, dataset_path)
            if check_other_pc_login(logon_data):
                other_pc_users.add(user_id)
                print(user_id)
    return other_pc_users

get_other_pc_users('Insider threat dataset\\r5.2')

JBI1134
JBI1134
JBI1134
JBI1134
HMI1448
JBI1134
JBI1134
KMC1934
JBI1134
HBP0009
OCW1127
DCB0446
JBI1134
OHR0583
JBI1134
RDC0128
LAS0129
LIJ1581
BCC1597
AKS0586
VRP0267
AHN0681
EMR0269
DSB0530
VPA0974
JRS0674
TKH1718
MRW0625
SJG0228
IRH1224
IQT0352
MKH0260
DSB0530
BUE1561
EMR0269
AHN0681
ELT1370
JMM0613
KDC1240
GVM1337
LKM0230
SEL1062
ESH1028
IBR0131
HLG1851
HMW0602
NMF1088
HTM1498
EMR0269
JDB1163
AHN0681
ACJ1797
GAT1343
XNC1338
YJB0171
BGR0917
HRR1154
LNS0317
SKW0066
KJM0889
DDR1876
ILM1441
LDM0399
MAG0780
ACC0950
HMC1847
ESM1828
PJB0886
KHG0205
DHS0204
AFC0792
ARD0790
DEO1964
NMP0782
DSB0530
ESH1028
ESH1028
VPA0974
HMC1677
VPA0974
XNC1338
BYV1695
DCR1648
XNC1338
ILM1441
KAD0387
DCA0857
BDF0015
KIS1205
BJM1314
SKC0074
FMG1549
ILM1441
LIJ1581
HST0084
LRV1554
KRC0098


KeyboardInterrupt: 